# Pybind11 examples of EjfatURI class

First import the built library "e2sar_py". Make sure the module is included in the path. Here we use the absolute path.

In [1]:
import sys

# IMPORTANT: Adjust the path to your built Python module as necessary
sys.path.append(
    '/home/xmei/Documents/hpdf_projects/E2SAR/build/src/pybind')

import e2sar_py

print(e2sar_py.get_version())

0.1.0b1


In [2]:

# Print to check the attributes
# print(dir(e2sar_py))

## Python helper class "IPAddress"
In the top module, we create a Python class `IPAddress` to bridge the Python string and the C++ boost IP class.

In [3]:
ip_instance = e2sar_py.IPAddress.from_string("192.168.1.3")
print(f"The IP address is {ip_instance}")
print(f"The IP address is of IPv4: {ip_instance.is_v4()}")
print(f"The IP address is of IPv6: {ip_instance.is_v6()}")

The IP address is 192.168.1.3
The IP address is of IPv4: True
The IP address is of IPv6: False


## Python class "EjfatURI"

In [4]:
# print(dir(e2sar_py.EjfatURI))   # print the supported methods

Create an "EjfatURI" object.

In [5]:
CP_IPV4 = "192.188.29.6"
CP_PORT = "18020"
LB_ID = "36"
SYNC_IPV4 = CP_IPV4
SYNC_PORT = "10920"
DP_IPV4 = "192.188.29.20"
DEMO_URI = f"ejfat://token@{CP_IPV4}:{CP_PORT}/lb/{LB_ID}?sync={SYNC_IPV4}:{SYNC_PORT}&data={DP_IPV4}"

try:
    uri = e2sar_py.EjfatURI(DEMO_URI)
    print(f"Created EjfatURI object successfully: {uri}")
except:
    print("Failed to create the EjfatURI object.")

Created EjfatURI object successfully: ejfat://token@192.188.29.6:18020/lb/36?sync=192.188.29.6:10920&data=192.188.29.20


In [6]:
# Print the EjfatURI object as a normal string
print(type(uri), uri)
print(type(uri.to_string()), uri.to_string())

<class 'e2sar_py.EjfatURI'> ejfat://token@192.188.29.6:18020/lb/36?sync=192.188.29.6:10920&data=192.188.29.20
<class 'str'> ejfat://token@192.188.29.6:18020/lb/36?sync=192.188.29.6:10920&data=192.188.29.20


The Python atrributes can be print out directly:

In [7]:
print("The created ejfatURI object instance:")
print(f"  useTls: {uri.get_useTls()}")
print(f"  lb_id: {uri.lb_id}")
print(f"  lb_name: {uri.lb_name if uri.lb_name else 'UNSET'}")
print(f"  session_id: {uri.session_id if uri.session_id else 'UNSET'}")


assert(uri.lb_id ==  LB_ID)

The created ejfatURI object instance:
  useTls: False
  lb_id: 36
  lb_name: UNSET
  session_id: UNSET


The private variables `lbName`, `lbId` and `sessionId` of the C++ `EjfatURI` class are mapped as Python attributes `EjfatURI.lb_name`, `EjfatURI.lb_id` and `EjfatURI.session_id`. The Python way to call their C++ get&set methods is demonstrated as below.

In [8]:
LB_NAME = "py_lb"

print(uri.lb_name)  # print nothing at the 1st run
if (uri.lb_name):
    print(f"Object's LB name: {uri.lb_name}") 
else:
    print("Object's LB name not set!")

print(f"Set LB name as {LB_NAME}")
uri.lb_name = LB_NAME   # equal to EjfatURI::set_LbName
print(f"Object's LB name: {uri.lb_name}")  # equal to EjfatURI::get_LbName
assert uri.lb_name == LB_NAME


Object's LB name not set!
Set LB name as py_lb
Object's LB name: py_lb


Set EjfatURI's sync or data plane's IP addresses and ports with Python tuples.

In [9]:
# Set syncAddr
sync_addr = (e2sar_py.IPAddress.from_string("192.168.1.1"), 8080)
uri.set_sync_addr(sync_addr)

# Set dataAddr
data_addr_v4 = (e2sar_py.IPAddress.from_string("10.0.0.1"), 9090)
uri.set_data_addr(data_addr_v4)

# Can only print bool varables directly
print("The EjfatURI instance:")
print(f"  has data plane address: {uri.has_data_addr()}")
print(f"  has sync plane address: {uri.has_sync_addr()}")
print(f"  has data plane IPv4 address: {uri.has_data_addr_v4()}")
print(f"  has data plane IPv6 address: {uri.has_data_addr_v6()}")

The EjfatURI instance:
  has data plane address: True
  has sync plane address: True
  has data plane IPv4 address: True
  has data plane IPv6 address: False


### C++ `Result<T>` return type bindings

In the `e2sar` C++ library, for an easier error detection, many methods are with return types `result<T>`. Their Python bindings are also of Object types with 4 atrributes inherit the C++ `has_value()`, `has_error()`, `value()` and `error()`. The below code demonstrates their Python usage.

In [10]:
print(uri.get_instance_token())  # the return type is an object

# Bindings where the return type is `result<std::string>`
def get_inst_token(uri : e2sar_py.EjfatURI):
    res = uri.get_instance_token()
    if (res.has_error()):
        print(f"Get instance token error: {res.error()}")
    elif (res.value() == ""):
        print("Instance Token Unset!")
    else:
        print("Instance Token:", res.value())

uri.set_instance_token("token123")
get_inst_token(uri)  # print the token value



Instance Token: token123


In [11]:

# Get control plane address
# Bindings where the return type is of `result<std::pair<ip::address, uint16_t>>`

def get_cp_address(uri : e2sar_py.EjfatURI):
    res = uri.get_cp_addr()
    if res.has_error():
        print(f"Get control plane address error: {res.error()}")
    elif (not res.value()):
        print(f"Control plane address UNSET!")
    else:
        print(f"get_cp_address return type is {type(res.value())}")
        print(f"  tuple[0] is of type: {type(res.value()[0])}")
        print(f"  tuple[1] is of type: {type(res.value()[1])}")
        return (str(res.value()[0]), res.value()[1])

get_cp_address(uri)

get_cp_address return type is <class 'tuple'>
  tuple[0] is of type: <class 'e2sar_py.IPAddress'>
  tuple[1] is of type: <class 'int'>


('192.188.29.6', 18020)

In [12]:
# Redo the above with a Python decorator of C++ return type result<std::pair<ip::address, uint16_t>>
def result_tuple2_handler(get_func):
    def wrapper(_obj):
        res1 = ""
        res2 = -1  # int object
        res = get_func(_obj)
        if res.has_error():
            print(f"{get_func.__name__} error: {res.error()}")
        elif (not res.value()[0]):
            print(f"{get_func.__name__} UNSET!")
        else:
            print(f"C++ {get_func.__name__} return type is {type(res.value())}")
            print(f"  tuple[0] is of type: {type(res.value()[0])}")
            print(f"  tuple[1] is of type: {type(res.value()[1])}")
            res1, res2 = str(res.value()[0]), res.value()[1]
        return (res1, res2)
    return wrapper

@result_tuple2_handler
def get_dp_address_v4(uri : e2sar_py.EjfatURI):
    return uri.get_data_addr_v4()

@result_tuple2_handler
def get_dp_address_v6(uri : e2sar_py.EjfatURI):
    return uri.get_data_addr_v6()

@result_tuple2_handler
def get_sync_address(uri : e2sar_py.EjfatURI):
    return uri.get_sync_addr()

dp_uri_v4, dp_port = get_dp_address_v4(uri)
print(f"Data plane IPV4 address: {dp_uri_v4}")
print(f"Data plane port: {dp_port}")

print()
print(f"Try get data plane IPV6 address: {get_dp_address_v6(uri)}")

print()
sync_uri, sync_port = get_sync_address(uri)
print(f"Sync IPV4 address: {sync_uri}")
print(f"Sync port: {sync_port}")

C++ get_dp_address_v4 return type is <class 'tuple'>
  tuple[0] is of type: <class 'e2sar_py.IPAddress'>
  tuple[1] is of type: <class 'int'>
Data plane IPV4 address: 10.0.0.1
Data plane port: 19522

get_dp_address_v6 error: <E2SARErrorInfo(code=4, message='Data plane address not available')>
Try get data plane IPV6 address: ('', -1)

C++ get_sync_address return type is <class 'tuple'>
  tuple[0] is of type: <class 'e2sar_py.IPAddress'>
  tuple[1] is of type: <class 'int'>
Sync IPV4 address: 192.168.1.1
Sync port: 8080


In [13]:
# The decorator applies to C++ return type of result<std::pair<std::string, u_int16_t>>
@result_tuple2_handler
def get_cp_hostname(uri : e2sar_py.EjfatURI):
    return uri.get_cp_host()

# Control plane host
cp_host, cp_host_port = get_cp_hostname(uri)
print(f"Control plane hostname: {cp_host}")
print(f"Control plane port: {'UNSET' if cp_host_port == -1 else cp_host_port}")

get_cp_hostname error: <E2SARErrorInfo(code=4, message='Control plane hostname not available')>
Control plane hostname: 
Control plane port: UNSET


### Examples taken from the C++ unit test `e2sar_uri_test`

1. `URITest1_1`: Create an EjfatURI with IPV6 address.

In [14]:
DEMO_IPV6 = "ejfat://[2001:4860:0:2001::68]:18020/lb/36?data=[2001:4860:0:2021::68]&sync=[2001:4860:0:2031::68]:19020"

uri_ipv6 = e2sar_py.EjfatURI(DEMO_IPV6)

assert(uri_ipv6.has_data_addr_v6() == True)
assert(uri_ipv6.has_data_addr_v4() == False)

print(f"Control plane (address, port) : {get_cp_address(uri_ipv6)}")
print(f"Data plane (address, port) : {get_dp_address_v6(uri_ipv6)}")
print(f"Sync (address, port) : {get_sync_address(uri_ipv6)}")

get_cp_address return type is <class 'tuple'>
  tuple[0] is of type: <class 'e2sar_py.IPAddress'>
  tuple[1] is of type: <class 'int'>
Control plane (address, port) : ('2001:4860:0:2001::68', 18020)
C++ get_dp_address_v6 return type is <class 'tuple'>
  tuple[0] is of type: <class 'e2sar_py.IPAddress'>
  tuple[1] is of type: <class 'int'>
Data plane (address, port) : ('2001:4860:0:2021::68', 19522)
C++ get_sync_address return type is <class 'tuple'>
  tuple[0] is of type: <class 'e2sar_py.IPAddress'>
  tuple[1] is of type: <class 'int'>
Sync (address, port) : ('2001:4860:0:2031::68', 19020)


2. `URITest2`: Set an EjfatURI object with IPv6 address and make sure the initilization is as expected.

In [15]:
DEMO_IPV4 = f"ejfat://token@192.188.29.6:18020/lb/{LB_ID}?sync=192.188.29.6:19020&data=192.188.29.20"

uri_ipv4 = e2sar_py.EjfatURI(DEMO_IPV4)

assert(uri_ipv4.lb_id == LB_ID)
assert(uri_ipv4.get_admin_token().value() == "token")
assert(str(uri_ipv4.get_cp_addr().value()[0]) == "192.188.29.6")
assert(uri_ipv4.get_cp_addr().value()[1] == 18020)
assert(str(uri_ipv4.get_data_addr_v4().value()[0]) == "192.188.29.20")
assert(uri_ipv4.get_data_addr_v4().value()[1] == 19522)
assert(str(uri_ipv4.get_sync_addr().value()[0]) == "192.188.29.6")
assert(uri_ipv4.get_sync_addr().value()[1] == 19020)

The Python bindings for C++ operators `==` and `!=`.

In [16]:
assert(uri_ipv4 != uri_ipv6)

uri_ipv4_2 = e2sar_py.EjfatURI(DEMO_IPV4)
assert(uri_ipv4_2 == uri_ipv4)

In [17]:
# URI without data plane address
DEMO_IPV4 = "ejfat://token@192.188.29.6:18020/lb/36?sync=192.188.29.6:19020"

uri_ipv4 = e2sar_py.EjfatURI(DEMO_IPV4)
assert(uri_ipv4.has_data_addr_v4() == False)
assert(uri_ipv4.has_data_addr_v6() == False)
assert(uri_ipv4.lb_id == LB_ID)
assert(uri_ipv4.get_admin_token().value() == "token")
assert(str(uri_ipv4.get_cp_addr().value()[0]) == "192.188.29.6")
assert(uri_ipv4.get_cp_addr().value()[1] == 18020)
assert(str(uri_ipv4.get_sync_addr().value()[0]) == "192.188.29.6")
assert(uri_ipv4.get_sync_addr().value()[1] == 19020)

In [18]:
# URI with control plane address only
DEMO_IPV4 = "ejfat://token@192.188.29.6:18020/lb/36"

uri_ipv4 = e2sar_py.EjfatURI(DEMO_IPV4)
assert(uri_ipv4.has_data_addr() == False)
assert(uri_ipv4.has_sync_addr() == False)
assert(uri_ipv4.lb_id == LB_ID)
assert(uri_ipv4.get_admin_token().value() == "token")
assert(str(uri_ipv4.get_cp_addr().value()[0]) == "192.188.29.6")
assert(uri_ipv4.get_cp_addr().value()[1] == 18020)

In [19]:
# Addresses without lb_id
DEMO_IPV4 = "ejfat://token@192.188.29.6:18020/"

uri_ipv4 = e2sar_py.EjfatURI(DEMO_IPV4)
assert(uri_ipv4.has_data_addr() == False)
assert(uri_ipv4.has_sync_addr() == False)
assert(uri_ipv4.lb_id == "")
assert(uri_ipv4.get_admin_token().value() == "token")
assert(str(uri_ipv4.get_cp_addr().value()[0]) == "192.188.29.6")
assert(uri_ipv4.get_cp_addr().value()[1] == 18020)

DEMO_IPV4 = "ejfat://token@192.188.29.6:18020"
uri_ipv4 = e2sar_py.EjfatURI(DEMO_IPV4)
assert(uri_ipv4.has_data_addr() == False)
assert(uri_ipv4.has_sync_addr() == False)
assert(uri_ipv4.lb_id == "")
assert(uri_ipv4.get_admin_token().value() == "token")
assert(str(uri_ipv4.get_cp_addr().value()[0]) == "192.188.29.6")
assert(uri_ipv4.get_cp_addr().value()[1] == 18020)

DEMO_IPV4 = "ejfat://token@192.188.29.6:18020/?sync=192.188.29.6:19020"
uri_ipv4 = e2sar_py.EjfatURI(DEMO_IPV4)
assert(uri_ipv4.has_data_addr() == False)
assert(uri_ipv4.has_sync_addr() == True)
assert(uri_ipv4.lb_id == "")
assert(uri_ipv4.get_admin_token().value() == "token")
assert(str(uri_ipv4.get_sync_addr().value()[0]) == "192.188.29.6")
assert(uri_ipv4.get_sync_addr().value()[1] == 19020)
assert(str(uri_ipv4.get_cp_addr().value()[0]) == "192.188.29.6")
assert(uri_ipv4.get_cp_addr().value()[1] == 18020)

3. `URITest3`: Exception when creating an EjfatURI object.

In [20]:
DEMO_WRONG_URI = "ejfact://token@192.188.29.6:18020/lb/36?sync=192.188.29.6:19020&data=192.188.29.20"

try:
    uri = e2sar_py.EjfatURI(DEMO_WRONG_URI)
except Exception as e:
    print(f"An error occured: {e}")


An error occured: Caught an unknown exception!


4. `URITest4`: create an EjfatURI object from an env var `EJFAT_URI`.

In [21]:
import os

os.environ['EJFAT_URI'] = DEMO_URI

assert(os.environ['EJFAT_URI'] == DEMO_URI)
res = e2sar_py.EjfatURI.get_from_env()  # Read from env_var "EJFAT_URI" by default

assert(res.has_error() == False)
assert(res.value().to_string() == DEMO_URI)

5. `URITest5`: Create an EjfatURI object from env vars other than `EJFAT_URI`.

In [22]:
# Delete env var EJFAT_URI
if 'EJFAT_URI' in os.environ:
    del os.environ['EJFAT_URI']
res = e2sar_py.EjfatURI.get_from_env()
assert(res.has_error() == True)

assert(res.error().code == e2sar_py.E2SARErrorc.Undefined)
print(res.error().message)

Environment variable EJFAT_URI not defined.


In [23]:
os.environ['EJFAT_URI_NEW'] = DEMO_URI
res = e2sar_py.EjfatURI.get_from_env('EJFAT_URI_NEW')

assert(res.has_error() == False)
assert(res.value().to_string() == DEMO_URI)

6. `URITest9`: Create EjfatURI object with `useTls`.

In [24]:
# URI with TLS
DEMO_IPV4 = "ejfats://192.188.29.6:18020/lb/36?sync=192.188.29.6:19020"

uri_ipv4 = e2sar_py.EjfatURI(DEMO_IPV4)
assert(uri_ipv4.get_useTls() == True)

7. `URITest10`, `URITest11`: Create EjfatURI object with Control Plane hostname.

In [25]:
DEMO_HOST_URI = "ejfats://ejfat-lb.es.net:18020/lb/36?sync=192.188.29.6:19020"

uri = e2sar_py.EjfatURI(DEMO_HOST_URI)
assert(uri.get_useTls() == True)
assert(uri.get_cp_host().value()[0] == 'ejfat-lb.es.net')
assert(uri.get_cp_addr().value()[0].is_v4() == True)

In [26]:
# Initialize with IPv6 address = True
uri = e2sar_py.EjfatURI(DEMO_HOST_URI, e2sar_py.EjfatURI.TokenType.admin, True)
assert(uri.get_useTls() == True)
assert(uri.get_cp_host().value()[0] == 'ejfat-lb.es.net')
assert(uri.get_cp_addr().value()[0].is_v4() == False)
assert(uri.get_cp_addr().value()[0].is_v6() == True)

9. `URITest12`: Create EjfatURI object with `session_id`.

In [27]:
# URI with session_id
DEMO_IPV4 = "ejfats://ejfat-lb.es.net:18020/lb/36?sync=192.188.29.6:19020&sessionid=mysessionid"

uri_ipv4 = e2sar_py.EjfatURI(DEMO_IPV4)
assert(uri_ipv4.session_id == "mysessionid")

10. `URITest13`: Create an EjfatURI object with customized data plane port.

In [28]:
# URI with customized data plane port (other than 19522)
DEMO_IPV4 = "ejfat://192.188.29.6:18020/lb/36?data=192.188.29.6:19020"

uri_ipv4 = e2sar_py.EjfatURI(DEMO_IPV4)
assert(str(uri_ipv4.get_data_addr_v4().value()[0]) == "192.188.29.6")
assert(uri_ipv4.get_data_addr_v4().value()[1] == 19020)